In [ ]:
!pip install pdfkit reportlab google-cloud-storage langchain langchain-community langchain-google-vertexai google-cloud-bigquery google-cloud-bigquery-storage pandas pyarrow google-auth db-dtypes google-cloud-aiplatform

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.cloud import aiplatform
from langchain.prompts import PromptTemplate
from langchain_google_vertexai import VertexAI
import html
from datetime import date, datetime
from IPython.display import HTML, display
import re
from io import BytesIO
import io
import decimal


In [44]:
# Configuration
PROJECT_ID = 'ra-development'
DATASET_ID = 'fivetran_demo_xero_xero'
LOCATION = 'europe-west2'
MODEL_NAME = 'gemini-1.5-flash-001'

# Utility Functions

def decimal_to_float(value):
  """Convert Decimal to float."""
  return float(value) if isinstance(value, Decimal) else value

def safe_percentage(part, whole):
  """Safely calculate percentage."""
  return (part / whole * 100) if whole != 0 else 0

def format_currency(value):
  """Format value as currency."""
  return f"£{value:,.2f}"

def calculate_percentage_change(current, previous):
  """Calculate percentage change."""
  if previous == 0:
    return float('inf') if current > 0 else float('-inf') if current < 0 else 0
  return (current - previous) / abs(previous) * 100

# Initialize clients
def initialize_clients():
  """Initialize BigQuery and Vertex AI clients."""
  bigquery_client = bigquery.Client()
  aiplatform.init(project=PROJECT_ID, location=LOCATION)
  llm = VertexAI(
    model_name=MODEL_NAME,
    max_output_tokens=2048,
    temperature=0, #A good starting point. This range encourages a focused, informative tone while allowing for some natural language variation
    top_p=0.9, #This narrower range prioritizes highly probable and relevant words, making the summary less prone to hallucinations or off-topic tangents.
    top_k=30, #Top-k sampling limits the vocabulary the model considers, but a narrower range might not be necessary for this task.
    project=PROJECT_ID,
    location=LOCATION,
    verbose=True,
  )
  return bigquery_client, llm

# Data retrieval functions
def fetch_data(client, query):
  """Fetch data from BigQuery."""
  return client.query(query).to_dataframe()

def get_pl_data(client):
  """Retrieve P&L data from BigQuery."""
  pl_query = f"""
  WITH
    report AS (
    SELECT
      date_month,
      CASE
        WHEN account_type = 'REVENUE' THEN 'Turnover'
        WHEN account_type = 'OVERHEADS' THEN 'Overheads'
        WHEN account_type = 'DIRECTCOSTS' THEN 'Direct Costs'
    END
      AS account_category,  -- Changed from account_report_category to account_category
      CASE
        WHEN account_type = 'REVENUE' THEN 1
        WHEN account_type = 'OVERHEADS' THEN 4
        WHEN account_type = 'DIRECTCOSTS' THEN 2
    END
      AS account_report_category_order,
      account_name AS account_group,
      account_id,
      account_code,
      account_class,
      account_type,
      COALESCE(SUM(net_amount* -1),0) AS net_amount
    FROM
      `{PROJECT_ID}.{DATASET_ID}.xero__profit_and_loss_report`
    GROUP BY
      1, 2, 3, 4, 5, 6, 7,8),
    spine AS (
    WITH
      rawdata AS (
      WITH
        p AS (
        SELECT
          0 AS generated_number
        UNION ALL
        SELECT
          1 ),
        unioned AS (
        SELECT
          p0.generated_number * POWER(2, 0) + p1.generated_number * POWER(2, 1) + p2.generated_number * POWER(2, 2) + p3.generated_number * POWER(2, 3) + p4.generated_number * POWER(2, 4) + p5.generated_number * POWER(2, 5) + p6.generated_number * POWER(2, 6) + 1 AS generated_number
        FROM
          p AS p0
        CROSS JOIN
          p AS p1
        CROSS JOIN
          p AS p2
        CROSS JOIN
          p AS p3
        CROSS JOIN
          p AS p4
        CROSS JOIN
          p AS p5
        CROSS JOIN
          p AS p6 )
      SELECT
        *
      FROM
        unioned
      WHERE
        generated_number <= 68
      ORDER BY
        generated_number ),
      all_periods AS (
      SELECT
        ( DATETIME_ADD( CAST( CAST('2023-06-01' AS date) AS datetime), INTERVAL ROW_NUMBER() OVER (ORDER BY 1) - 0 month ) ) AS date_month
      FROM
        rawdata ),
      filtered AS (
      SELECT
        *
      FROM
        all_periods
      WHERE
        date_month <= DATETIME_ADD( CAST( current_date AS datetime), INTERVAL 1 month ) )
    SELECT
      *
    FROM
      filtered ),
    cleaned AS (
    SELECT
      CAST(date_month AS date) AS date_month
    FROM
      spine ),
    calendar AS (
    SELECT
      *
    FROM
      cleaned),
    joined AS (
    SELECT
      calendar.date_month,
      report.account_id,
      report.account_group,
      report.account_code,
      report.account_class,
      report.account_type,
      report.account_category,  -- Changed from account_report_category to account_category
      report.account_report_category_order,
      COALESCE(SUM(net_amount * -1),0) AS net_amount
    FROM
      calendar
    LEFT JOIN
      report
    ON
      calendar.date_month = CAST( TIMESTAMP_TRUNC( CAST(report.date_month AS timestamp ), month ) AS date)
    GROUP BY
      1,2,3,4,5,6,7,8),
  report_with_subtotals_totals as (
  SELECT
    date_month,
    account_report_category_order,
    account_category,  -- Changed from account_report_category account_category to account_category
    SUM(net_amount) AS net_amount
  FROM
    joined
  WHERE
    account_category IS NOT NULL  -- Changed from account_report_category to account_category
  GROUP BY
    1, 2, 3
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      3 AS account_report_category_order,
      'Gross Profit' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs'))  -- Changed from account_report_category to account_category
  GROUP BY
    1, 2, 3, 4
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      5 AS account_report_category_order,
      'Net Profit' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs', 'Overheads'))  -- Changed from account_report_category to account_category
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      7 AS account_report_category_order,
      'Retained Earnings' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) * 0.75 AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs', 'Overheads'))  -- Changed from account_report_category to account_category
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      6 AS account_report_category_order,
      'Taxation' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) * 0.25 AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs', 'Overheads'))  -- Changed from account_report_category to account_category
  GROUP BY
    1, 2, 3, 4
  ORDER BY
    date_month,
    account_report_category_order)
  SELECT
    *
  FROM
   report_with_subtotals_totals
  GROUP BY
    ALL
  ORDER BY
   date_month,
   account_report_category_order
  """

  account_group_query = f"""
  WITH
  report AS (
  SELECT
    date_month,
    CASE
      WHEN account_type = 'REVENUE' THEN 'Turnover'
      WHEN account_type = 'OVERHEADS' THEN 'Overheads'
      WHEN account_type = 'DIRECTCOSTS' THEN 'Direct Costs'
  END
    AS account_category,
    CASE
      WHEN account_type = 'REVENUE' THEN 1
      WHEN account_type = 'OVERHEADS' THEN 4
      WHEN account_type = 'DIRECTCOSTS' THEN 2
  END
    AS account_report_category_order,
    account_name AS account_group,
    account_id,
    account_code,
    account_class,
    account_type,
    COALESCE(SUM(net_amount* -1),0) AS net_amount
  FROM
    `{PROJECT_ID}.{DATASET_ID}.xero__profit_and_loss_report`
  GROUP BY
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8),
  spine AS (
  WITH
    rawdata AS (
    WITH
      p AS (
      SELECT
        0 AS generated_number
      UNION ALL
      SELECT
        1 ),
      unioned AS (
      SELECT
        p0.generated_number * POWER(2, 0) + p1.generated_number * POWER(2, 1) + p2.generated_number * POWER(2, 2) + p3.generated_number * POWER(2, 3) + p4.generated_number * POWER(2, 4) + p5.generated_number * POWER(2, 5) + p6.generated_number * POWER(2, 6) + 1 AS generated_number
      FROM
        p AS p0
      CROSS JOIN
        p AS p1
      CROSS JOIN
        p AS p2
      CROSS JOIN
        p AS p3
      CROSS JOIN
        p AS p4
      CROSS JOIN
        p AS p5
      CROSS JOIN
        p AS p6 )
    SELECT
      *
    FROM
      unioned
    WHERE
      generated_number <= 68
    ORDER BY
      generated_number ),
    all_periods AS (
    SELECT
      ( DATETIME_ADD( CAST( CAST('2023-06-01' AS date) AS datetime), INTERVAL ROW_NUMBER() OVER (ORDER BY 1) - 0 month ) ) AS date_month
    FROM
      rawdata ),
    filtered AS (
    SELECT
      *
    FROM
      all_periods
    WHERE
      date_month <= DATETIME_ADD( CAST( current_date AS datetime), INTERVAL 1 month ) )
  SELECT
    *
  FROM
    filtered ),
  cleaned AS (
  SELECT
    CAST(date_month AS date) AS date_month
  FROM
    spine ),
  calendar AS (
  SELECT
    *
  FROM
    cleaned),
  joined AS (
  SELECT
    calendar.date_month,
    report.account_id,
    report.account_group,
    report.account_code,
    report.account_type,
    report.account_class,
    report.account_category,
    COALESCE(SUM(net_amount * -1),0) AS net_amount
  FROM
    calendar
  LEFT JOIN
    report
  ON
    calendar.date_month = CAST( TIMESTAMP_TRUNC( CAST(report.date_month AS timestamp ), month ) AS date)
  GROUP BY
    1,2,3,4,5,6,7)
select * from joined
group by all
order by date_month, account_code
"""

  transactions_query = f"""
  WITH
    spine AS (
    WITH
      rawdata AS (
      WITH
        p AS (
        SELECT
          0 AS generated_number
        UNION ALL
        SELECT
          1 ),
        unioned AS (
        SELECT
          p0.generated_number * POWER(2, 0) + p1.generated_number * POWER(2, 1) + p2.generated_number * POWER(2, 2) + p3.generated_number * POWER(2, 3) + p4.generated_number * POWER(2, 4) + p5.generated_number * POWER(2, 5) + p6.generated_number * POWER(2, 6) + 1 AS generated_number
        FROM
          p AS p0
        CROSS JOIN
          p AS p1
        CROSS JOIN
          p AS p2
        CROSS JOIN
          p AS p3
        CROSS JOIN
          p AS p4
        CROSS JOIN
          p AS p5
        CROSS JOIN
          p AS p6 )
      SELECT
        *
      FROM
        unioned
      WHERE
        generated_number <= 68
      ORDER BY
        generated_number ),
      all_periods AS (
      SELECT
        ( DATETIME_ADD( CAST( CAST('2019-01-01' AS date) AS datetime), INTERVAL ROW_NUMBER() OVER (ORDER BY 1) - 0 month ) ) AS date_month
      FROM
        rawdata ),
      filtered AS (
      SELECT
        *
      FROM
        all_periods
      WHERE
        date_month <= DATETIME_ADD( CAST( current_date AS datetime), INTERVAL 1 month ) )
    SELECT
      *
    FROM
      filtered ),
    cleaned AS (
    SELECT
      CAST(date_month AS date) AS date_month
    FROM
      spine ),
    calendar AS (
    SELECT
      *
    FROM
      cleaned),
    ledger AS (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_ID}.xero__general_ledger` ),
    joined AS (
    SELECT
      TO_HEX(MD5(CAST(COALESCE(CAST(calendar.date_month AS string ), '') || '-' || COALESCE(CAST(ledger.account_id AS string ), '') AS string ))) AS profit_and_loss_pk,
      calendar.date_month,
      ledger.account_id,
      ledger.account_name as account_group,
      ledger.account_code,
      ledger.account_type,
      COALESCE(SUM(ledger.net_amount * -1),0) AS net_amount
    FROM
      calendar
    LEFT JOIN
      ledger
    ON
      calendar.date_month = CAST( TIMESTAMP_TRUNC( CAST(ledger.journal_date AS timestamp ), month ) AS date)
    WHERE
      ledger.account_class IN ('REVENUE',
        'EXPENSE')
    GROUP BY
      1,
      2,
      3,
      4,
      5,
      6),
    journals AS (
    SELECT
      DATE_TRUNC(journal_date,MONTH) AS date_month,
      account_code,
      journal_number,
      reference,
      source_type,
      account_name as account_group,
      account_type,
      description,
      gross_amount,
      net_amount,
      tax_amount
    FROM
      `{PROJECT_ID}.{DATASET_ID}.xero__general_ledger` ),
    account_groups AS (
    SELECT
      date_month,
      account_group,
      account_code,
      account_type,
      SUM(net_amount) AS net_amount
    FROM
      joined
    GROUP BY
      1,
      2,
      3,
      4)
  SELECT
    g.date_month,
    g.account_group,
    g.account_type,
    j.account_code,
    j.description,
    COALESCE(j.net_amount * -1,0) AS net_amount
  FROM
    account_groups g
  LEFT JOIN
    journals j
  ON
    g.account_code = j.account_code
    AND g.date_month = j.date_month
  ORDER BY
    1,
    2,
    3,
    4,
    5
"""
  data = fetch_data(client, pl_query)
  account_group_data = fetch_data(client, account_group_query)
  transactions_data = fetch_data(client, transactions_query)
  return data, account_group_data, transactions_data

# Analysis functions

def get_significant_transactions(transactions_data, account_group, date, threshold=0.1):
  current_month = transactions_data[(transactions_data['account_group'] == account_group) &
                   (transactions_data['date_month'] == date)]
  total_amount = current_month['net_amount'].sum()
  significant_transactions = current_month[abs(current_month['net_amount']) > abs(total_amount * threshold)]
  return significant_transactions.to_dict('records')

def calculate_percentage_change(current, previous):
  """Calculate percentage change, treating costs as positive values."""
  current_abs = abs(current)
  previous_abs = abs(previous)
  if previous_abs == 0:
    return float('inf') if current_abs > 0 else 0
  return ((current_abs - previous_abs) / previous_abs) * 100

def format_financial_change(current, previous, category_name):
  """Format financial change consistently, treating costs as positive values."""
  current_abs = abs(current)
  previous_abs = abs(previous)
  change = current_abs - previous_abs
  percentage_change = calculate_percentage_change(current, previous)

  if change > 0:
    change_direction = "increased"
  elif change < 0:
    change_direction = "decreased"
  else:
    change_direction = "remained unchanged"

  return f"{category_name} {change_direction} to £{current_abs:,.2f} in the current month, " \
      f"from £{previous_abs:,.2f} in the previous month, " \
      f"a change of {abs(percentage_change):.2f}%."

def analyze_cost_changes(transactions_data, account_group, date):
  current_month = transactions_data[(transactions_data['account_group'] == account_group) &
                   (transactions_data['date_month'] == date)]
  previous_month = transactions_data[(transactions_data['account_group'] == account_group) &
                    (transactions_data['date_month'] == date - pd.DateOffset(months=1))]

  current_total = current_month['net_amount'].sum()
  previous_total = previous_month['net_amount'].sum()
  total_change = abs(current_total) - abs(previous_total)

  category_changes = {}
  for category in ['Salary', 'Bonus', 'Dividend', 'Software/Subscription', 'Other']:
    current_category = current_month[current_month['description'].str.contains(category, case=False, na=False)]
    previous_category = previous_month[previous_month['description'].str.contains(category, case=False, na=False)]

    current_amount = current_category['net_amount'].sum()
    previous_amount = previous_category['net_amount'].sum()
    change = abs(current_amount) - abs(previous_amount)

    category_changes[category] = {
      'current_amount': current_amount,
      'previous_amount': previous_amount,
      'change': change,
      'percentage_change': calculate_percentage_change(current_amount, previous_amount),
      'formatted_change': format_financial_change(current_amount, previous_amount, category)
    }

  return {
    'total_change': total_change,
    'total_percentage_change': calculate_percentage_change(current_total, previous_total),
    'formatted_total_change': format_financial_change(current_total, previous_total, account_group),
    'category_changes': category_changes
  }

def analyze_overhead_trends(data, months=6):
  # Filter data for Overheads category and last 6 months
  overhead_data = data[(data['account_category'] == 'Overheads') &
             (data['date_month'] >= data['date_month'].max() - pd.DateOffset(months=months))]

  # Group by account group and calculate month-over-month growth
  grouped = overhead_data.groupby(['account_group', 'date_month'])['net_amount'].sum().unstack()
  growth_rates = grouped.pct_change(axis=1).mean(axis=1)

  # Identify groups with significant growth (e.g., more than 10% average monthly growth)
  significant_growth = growth_rates[growth_rates > 0.10]

  return significant_growth

def identify_cancelling_transactions(transactions):
  """
  Identify pairs of transactions that cancel each other out.
  """
  cancelling_pairs = []
  for i, trans1 in enumerate(transactions):
    for j, trans2 in enumerate(transactions[i+1:], start=i+1):
      if (trans1['account_group'] == trans2['account_group'] and
        trans1['description'] == trans2['description'] and
        trans1['net_amount'] == -trans2['net_amount']):
        cancelling_pairs.append((i, j))
  return cancelling_pairs

def get_account_group_details(account_group_data, transactions_data, category, date):
  current_month = account_group_data[(account_group_data['account_type'] == category) &
                    (account_group_data['date_month'] == date)]
  previous_month = account_group_data[(account_group_data['account_type'] == category) &
                    (account_group_data['date_month'] == date - pd.DateOffset(months=1))]

  group_totals = current_month.groupby('account_group')['net_amount'].sum().to_dict()
  prev_group_totals = previous_month.groupby('account_group')['net_amount'].sum().to_dict()

  group_details = {}
  for group, amount in group_totals.items():
    previous_amount = prev_group_totals.get(group, 0)
    significant_transactions = get_significant_transactions(transactions_data, group, date)

    group_details[group] = {
      'total': amount,
      'previous_total': previous_amount,
      'change': amount - previous_amount,
      'percentage_change': calculate_percentage_change(amount, previous_amount),
      'is_new': group not in prev_group_totals,
      'significant_transactions': significant_transactions
    }

  if category == 'EXPENSE':
    # Add trend analysis
    trend_analysis = analyze_overhead_trends(account_group_data)
    return {
      'group_details': group_details,
      'trend_analysis': trend_analysis.to_dict()
    }
  elif category in ['REVENUE', 'DIRECTCOSTS']:
    return group_details
  else:
    # For calculated categories, just return the total
    total = current_month['net_amount'].sum()
    previous_total = previous_month['net_amount'].sum()
    return {
      'total': total,
      'previous_total': previous_total,
      'change': total - previous_total,
      'percentage_change': calculate_percentage_change(total, previous_total)
    }

# Data Preparation

def process_data(data, account_group_data, transactions_data):
    data['date_month'] = pd.to_datetime(data['date_month'], format='%Y-%m-%d')
    data['net_amount'] = data['net_amount'].astype(float)

    unique_months = sorted(data['date_month'].unique())
    last_month = unique_months[-1]
    second_last_month = unique_months[-2]
    third_last_month = unique_months[-3]

    current_month_data = data[data['date_month'] == second_last_month]
    previous_month_data = data[data['date_month'] == third_last_month]
    two_months_ago_data = data[data['date_month'] == unique_months[-4]]

    current_year = second_last_month.year
    previous_year = current_year - 1

    ytd_data_current = data[(data['date_month'].dt.year == current_year) & (data['date_month'] <= second_last_month)]
    ytd_totals_current = ytd_data_current.groupby(['account_category', 'account_report_category_order'])['net_amount'].sum().reset_index()

    ytd_data_previous = data[(data['date_month'].dt.year == previous_year) & (data['date_month'] <= second_last_month.replace(year=previous_year))]
    ytd_totals_previous = ytd_data_previous.groupby(['account_category', 'account_report_category_order'])['net_amount'].sum().reset_index()

    report_data = current_month_data.merge(previous_month_data, on=['account_category', 'account_report_category_order'], suffixes=('', '_prev'), how='outer')
    report_data = report_data.merge(two_months_ago_data, on=['account_category', 'account_report_category_order'], suffixes=('', '_2mo_ago'), how='outer')
    report_data = report_data.merge(ytd_totals_current, on=['account_category', 'account_report_category_order'], suffixes=('', '_ytd'), how='outer')
    report_data = report_data.merge(ytd_totals_previous, on=['account_category', 'account_report_category_order'], suffixes=('', '_prev_ytd'), how='outer')

    report_data['pct_change_mom'] = (report_data['net_amount'] - report_data['net_amount_prev']) / abs(report_data['net_amount_prev'])
    report_data['pct_change_ytd'] = (report_data['net_amount_ytd'] - report_data['net_amount_prev_ytd']) / abs(report_data['net_amount_prev_ytd'])

    # Process the account group data
    account_group_data['date_month'] = pd.to_datetime(account_group_data['date_month'], format='%Y-%m-%d')
    account_group_data['net_amount'] = account_group_data['net_amount'].astype(float)

    # Process the transactions data
    transactions_data['date_month'] = pd.to_datetime(transactions_data['date_month'], format='%Y-%m-%d')
    transactions_data['net_amount'] = transactions_data['net_amount'].astype(float)

    # Prepare data for LLM analysis
    last_month_data = report_data[report_data['date_month'] == second_last_month]
    last_month_data = last_month_data[['account_category', 'net_amount', 'net_amount_prev', 'net_amount_ytd', 'net_amount_prev_ytd']]

    return report_data, second_last_month, third_last_month, unique_months, current_year, previous_year

def prepare_account_group_details(current_month_groups):
  account_group_details = ""
  for category in current_month_groups:
    details = current_month_groups[category]
    account_group_details += f"\n{category}:\n"

    if category == 'EXPENSE':
      for group, group_data in details['group_details'].items():
        current_amount = group_data['total']
        previous_amount = group_data['previous_total']
        change = group_data['change']
        percentage_change = group_data['percentage_change']
        is_new = group_data['is_new']

        if is_new:
          account_group_details += f" - NEW: {group}: {format_currency(current_amount)}\n"
        else:
          account_group_details += f" - {group}: {format_currency(current_amount)} (Previous: {format_currency(previous_amount)}, Change: {format_currency(change)}, {percentage_change:.2f}%)\n"

        if group_data['significant_transactions']:
          account_group_details += "  Significant Transactions:\n"
          for transaction in group_data['significant_transactions']:
            amount = transaction['net_amount']
            description = transaction['description']
            account_group_details += f"   * {format_currency(amount)} - {description}\n"

      account_group_details += "\nOverhead Trend Analysis (Past 6 Months):\n"
      for group, growth_rate in details['trend_analysis'].items():
        account_group_details += f" - {group}: {growth_rate:.2%} average monthly growth\n"

    elif category in ['REVENUE', 'DIRECTCOSTS']:
      for group, group_data in details.items():
        current_amount = group_data['total']
        previous_amount = group_data['previous_total']
        change = group_data['change']
        percentage_change = group_data['percentage_change']
        is_new = group_data['is_new']

        if is_new:
          account_group_details += f" - NEW: {group}: {format_currency(current_amount)}\n"
        else:
          account_group_details += f" - {group}: {format_currency(current_amount)} (Previous: {format_currency(previous_amount)}, Change: {format_currency(change)}, {percentage_change:.2f}%)\n"

        if group_data.get('significant_transactions'):
          account_group_details += "  Significant Transactions:\n"
          for transaction in group_data['significant_transactions']:
            amount = transaction['net_amount']
            description = transaction['description']
            account_group_details += f"   * {format_currency(amount)} - {description}\n"

    else: # For calculated categories like 'Gross Profit' and 'Retained Earnings'
      current_amount = details['total']
      previous_amount = details['previous_total']
      change = details['change']
      percentage_change = details['percentage_change']
      account_group_details += f" Current Month Total: {format_currency(current_amount)}\n"
      account_group_details += f" Previous Month Total: {format_currency(previous_amount)}\n"
      account_group_details += f" Change: {format_currency(change)} ({percentage_change:.2f}%)\n"

    account_group_details += "\n"

  return account_group_details

def generate_analysis_prompt(month, financial_data, account_group_details, cost_change_analysis):
  return PromptTemplate.from_template("""
  Analyze the following Profit and Loss data for {month} and provide a management report:

  {financial_data}

  Account Group Details (including significant transactions):
  {account_group_details}

  Cost Change Analysis:
  {cost_change_analysis}

  Do not use markdown formatting, * or #. All financial amounts are in GBP (£)

  **Focus on:**

  1. Analyze Revenue, Cost of Delivery, and Overheads, highlighting significant account groups and transactions that have influenced these categories.
  2. Pay special attention to the significant transactions listed for each account group. Explain how these transactions have impacted the overall performance of their respective categories.
  3. Explain how changes in these transactional categories, including the effect of significant transactions, have impacted Gross Margin, Net Profit, and Retained Earnings.
  4. Compare against previous month actual, calling out significant increases/decreases in the numbers, and relate these to any anomalous transactions where relevant.
  5. Analyze YTD vs. LYTD performance, highlighting areas of over- and under-performance, considering the impact of significant transactions on these trends.
  6. Ignore any double-entry but cancelling transactions, such as a negative expense followed by a positive expense of the same amount. These are typically corrections of administrative errors and should not be considered in the analysis.
  7. When analyzing cost changes, focus on the actual causes of increases or decreases as shown in the Cost Change Analysis. Do not attribute changes to new hires or costs unless explicitly stated in the analysis.
  8. Ensure all financial calculations are consistent. When discussing costs, always treat them as positive values for calculation purposes, even if they are recorded as negative values in the accounting system.
  9. When describing changes in costs or revenues, ensure that the direction of change (increase/decrease) matches the sign of the percentage change. For example, if a cost decreased, the percentage change should be described as a decrease (negative percentage), not an increase.
  10. Do not speculate about the reason any changes or transactions you see, and never state any person (e.g. staff) names when discussing salaries, bonuses or dividends:

  When analyzing significant transactions, provide context for each transaction:
  1. Indicate whether it's a new transaction that wasn't present in the previous months.
  2. If it's not new, specify how much it has changed compared to its average in previous months.
  3. Explain the potential reasons for new transactions or significant changes in existing ones.
  4. For recurring transactions, avoid describing them as "new" and instead focus on any changes in their amounts.

  When discussing changes in categories or specific expenses:
  1. Always include the current amount, previous amount, and the percentage change.
  2. For new expenses or categories, clearly state that they are new and provide their amounts.
  3. Use the following format for discussing changes:
   "[Category] [increased/decreased] to [current amount] in [current month], compared to [previous amount] in [previous month], a change of [percentage]%."
  4. When mentioning new expenses, use the format:
   "A new expense for [category] was introduced, amounting to [amount]."
  5. For recurring expenses that have changed, use the format:
   "The expense for [category] [increased/decreased] to [current amount], compared to its previous average of [average amount], a change of [percentage]%."

  ** Output your analysis in a concise and informative manner, suitable for senior management and executives. Format your response using the following HTML structure:**
  ** Use the Economist Style Guide as your writing style, with bold tags on important elements such as account group names and anomalous transaction details. Do not use markdown formatting or "**" in your responses.

  <h1>📊 Executive Summary</h1>
  <ul>
   <li>📈 [Key summary point about overall financial performance in the context of change vs. last month]</li>
   <li>🔮 [Key summary point about future outlook or recommendations, considering observed anomalies]</li>
  </ul>

  <h2>💰 Revenue</h2>
  <ul>
   <li>[Key point about revenue]</li>
   <li>[Explanation of significant variations in revenue account groups, including impact of notable transactions]</li>
  </ul>

  <h3>💼 Direct Costs</h3>
  <ul>
   <li>[Key point about direct costs]</li>
   <li>[Explanation of the top one or two significant variations in direct costs account groups, including impact of the one most notable transaction in each delivery group. For each significant transaction, specify whether it's new or how it has changed from its average in previous months.]</li>
   <li>[Analysis of the one most significant cost change, focusing on the actual causes of increases or decreases as shown in the Cost Change Analysis]</li>
  </ul>

  <h3>🏭 Gross Margin</h3>
  <ul>
   <li>[Key point about gross margin]</li>
   <li>[Explanation of how changes in Revenue and Cost of Delivery, including significant transactions, have impacted Gross Margin]</li>
  </ul>

  <h3>🛍️ Overheads</h3>
  <ul>
   <li>[Overview of Overheads, including total amount, change from previous month, and percentage change]</li>
   <li>[Highlight the one most Significant change in Overhead subcategories, with specific amount and percentage change]</li>
  </ul>

  <h2>🏦 Net Profit and Retained Earnings</h2>
  <ul>
   <li>[Key point about net profit and retained earnings]</li>
   <li>[Explanation of how changes in Revenue, direct costs and Overheads, including significant transactions, have impacted Net Profit and Retained Earnings]</li>
  </ul>

  Ensure that the Executive Summary provides a high-level overview of the most important points from your detailed analysis, including the impact of any significant or anomalous transactions.
  """)


def generate_financial_data_string(report_data, second_last_month, third_last_month, current_year, previous_year):
  """Generate a string containing financial data for the LLM prompt."""
  financial_data = f"""
  Last Month's Performance ({second_last_month.strftime('%B %Y')}):
  Revenue: {format_currency(report_data[report_data['account_category'] == 'Turnover']['net_amount'].values[0])}
  Gross Margin %: {safe_percentage(report_data[report_data['account_category'] == 'Gross Profit']['net_amount'].values[0], report_data[report_data['account_category'] == 'Turnover']['net_amount'].values[0]):.2f}%
  Retained Earnings: {format_currency(report_data[report_data['account_category'] == 'Retained Earnings']['net_amount'].values[0])}
  Direct Costs: {format_currency(report_data[report_data['account_category'] == 'Direct Costs']['net_amount'].values[0])}
  Overheads: {format_currency(report_data[report_data['account_category'] == 'Overheads']['net_amount'].values[0])}
  Net Profit: {format_currency(report_data[report_data['account_category'] == 'Net Profit']['net_amount'].values[0])}

  Previous Month's Performance ({third_last_month.strftime('%B %Y')}):
  Revenue: {format_currency(report_data[report_data['account_category'] == 'Turnover']['net_amount_prev'].values[0])}
  Gross Margin %: {safe_percentage(report_data[report_data['account_category'] == 'Gross Profit']['net_amount_prev'].values[0], report_data[report_data['account_category'] == 'Turnover']['net_amount_prev'].values[0]):.2f}%
  Retained Earnings: {format_currency(report_data[report_data['account_category'] == 'Retained Earnings']['net_amount_prev'].values[0])}
  Direct Costs: {format_currency(report_data[report_data['account_category'] == 'Direct Costs']['net_amount_prev'].values[0])}
  Overheads: {format_currency(report_data[report_data['account_category'] == 'Overheads']['net_amount_prev'].values[0])}
  Net Profit: {format_currency(report_data[report_data['account_category'] == 'Net Profit']['net_amount_prev'].values[0])}

  Year-to-Date Performance:
  Revenue: {format_currency(report_data[report_data['account_category'] == 'Turnover']['net_amount_ytd'].values[0])}
  Gross Margin %: {safe_percentage(report_data[report_data['account_category'] == 'Gross Profit']['net_amount_ytd'].values[0], report_data[report_data['account_category'] == 'Turnover']['net_amount_ytd'].values[0]):.2f}%
  Retained Earnings: {format_currency(report_data[report_data['account_category'] == 'Retained Earnings']['net_amount_ytd'].values[0])}
  Direct Costs: {format_currency(report_data[report_data['account_category'] == 'Direct Costs']['net_amount_ytd'].values[0])}
  Overheads: {format_currency(report_data[report_data['account_category'] == 'Overheads']['net_amount_ytd'].values[0])}
  Net Profit: {format_currency(report_data[report_data['account_category'] == 'Net Profit']['net_amount_ytd'].values[0])}

  Previous Year's Year-to-Date Performance:
  Revenue: {format_currency(report_data[report_data['account_category'] == 'Turnover']['net_amount_prev_ytd'].values[0])}
  Gross Margin %: {safe_percentage(report_data[report_data['account_category'] == 'Gross Profit']['net_amount_prev_ytd'].values[0], report_data[report_data['account_category'] == 'Turnover']['net_amount_prev_ytd'].values[0]):.2f}%
  Retained Earnings: {format_currency(report_data[report_data['account_category'] == 'Retained Earnings']['net_amount_prev_ytd'].values[0])}
  Direct Costs: {format_currency(report_data[report_data['account_category'] == 'Direct Costs']['net_amount_prev_ytd'].values[0])}
  Overheads: {format_currency(report_data[report_data['account_category'] == 'Overheads']['net_amount_prev_ytd'].values[0])}
  Net Profit: {format_currency(report_data[report_data['account_category'] == 'Net Profit']['net_amount_prev_ytd'].values[0])}
  """
  return financial_data

def generate_report_html(report_data, second_last_month, third_last_month, unique_months, current_year, previous_year):
  # Calculate revenue values
  revenue_current = report_data[report_data['account_category'] == 'Turnover']['net_amount'].values[0]
  revenue_prev = report_data[report_data['account_category'] == 'Turnover']['net_amount_prev'].values[0]
  revenue_2mo_ago = report_data[report_data['account_category'] == 'Turnover']['net_amount_2mo_ago'].values[0]
  revenue_ytd = report_data[report_data['account_category'] == 'Turnover']['net_amount_ytd'].values[0]
  revenue_prev_ytd = report_data[report_data['account_category'] == 'Turnover']['net_amount_prev_ytd'].values[0]

  report_data_html = f"""
  <h2>Profit & Loss Report Detail with Period-on-Period Comparisons</h2>
  <table class="financial-table">
    <tr>
      <th>Category</th>
      <th>{second_last_month.strftime('%b %Y')}</th>
      <th>% Change MoM</th>
      <th>{third_last_month.strftime('%b %Y')}</th>
      <th>{unique_months[-4].strftime('%b %Y')}</th>
      <th>YTD {current_year}</th>
      <th>% Change YTD</th>
      <th>YTD {previous_year}</th>
    </tr>
  """

  for _, row in report_data.iterrows():
    category = row['account_category']
    current_amount = row['net_amount']
    pct_change_mom = row['pct_change_mom']
    previous_amount = row['net_amount_prev']
    two_months_ago_amount = row['net_amount_2mo_ago']
    ytd_amount = row['net_amount_ytd']
    pct_change_ytd = row['pct_change_ytd']
    prev_ytd_amount = row['net_amount_prev_ytd']

    class_name = 'category'
    if category in ['Direct Costs', 'Overheads', 'Taxation']:
      class_name = 'subcategory'

    report_data_html += f"""
    <tr class="{class_name}">
      <td>{category}</td>
      <td>{format_currency(current_amount)}</td>
      <td>{pct_change_mom:.2%}</td>
      <td>{format_currency(previous_amount)}</td>
      <td>{format_currency(two_months_ago_amount)}</td>
      <td>{format_currency(ytd_amount)}</td>
      <td>{pct_change_ytd:.2%}</td>
      <td>{format_currency(prev_ytd_amount)}</td>
    </tr>
    """

    if category in ['Direct Costs', 'Gross Profit', 'Overheads', 'Net Profit', 'Retained Earnings']:
      current_percentage = safe_percentage(current_amount, revenue_current)
      prev_percentage = safe_percentage(previous_amount, revenue_prev)
      two_months_ago_percentage = safe_percentage(two_months_ago_amount, revenue_2mo_ago)
      ytd_percentage = safe_percentage(ytd_amount, revenue_ytd)
      prev_ytd_percentage = safe_percentage(prev_ytd_amount, revenue_prev_ytd)

      report_data_html += f"""
      <tr class="{class_name}-percentage">
        <td>{category} as % of Sales</td>
        <td>{current_percentage:.2f}%</td>
        <td></td>
        <td>{prev_percentage:.2f}%</td>
        <td>{two_months_ago_percentage:.2f}%</td>
        <td>{ytd_percentage:.2f}%</td>
        <td></td>
        <td>{prev_ytd_percentage:.2f}%</td>
      </tr>
      """

  report_data_html += "</table>"

  return report_data_html

def generate_full_html_output(report_analysis_html, report_data_html):
  """Generate the full HTML output for display in the notebook."""

  return f"""
  <html>
  <head>
  <style>
    body {{
      font-family: Arial, sans-serif;
      line-height: 1.6;
      color: #333;
    }}
    h1, h2, h3 {{
      color: #2c3e50;
    }}
    table {{
      border-collapse: collapse;
      width: 100%;
	margin-top: 20px;
    }}
    th, td {{
      border: 1px solid #ddd;
      padding: 8px;
      text-align: right;
    }}
    th {{
      background-color: #f2f2f2;
      color: #2c3e50;
    }}
    .category {{
      text-align: left;
      font-weight: bold;
    }}
    .subcategory {{
      padding-left: 20px;
    }}
    .percentage {{
      font-style: italic;
      color: #555;
    }}
    .analysis {{
      border: 1px solid #ddd;
      padding: 15px;
      margin-bottom: 20px;
    }}
    .analysis h2 {{
      color: #2c3e50;
      padding-bottom: 10px;
    }}
    .analysis h3 {{
      color: #2c3e50;
    }}
    .analysis ul {{
      padding-left: 0;
    }}
    .analysis li {{
      margin-bottom: 10px;
    }}
    .executive-summary li {{
      font-weight: bold;
    }}
   </style>
  </head>
  <body>
  <h1>📊 Profit and Loss Report</h1>
  {report_data_html}
  <div class="analysis">
  {report_analysis_html}
  </div>
  </body>
  </html>
  """

# Modify the write_report_to_bigquery function
def write_report_to_bigquery(client, date_month, report_data, report_analysis):
  """Write the generated report to BigQuery."""
  dataset_id = 'fivetran_demo_xero_xero'
  table_id = 'pl_reports_demo'

  dataset_ref = client.dataset(dataset_id)
  table_ref = dataset_ref.table(table_id)

  # Define the updated schema
  schema = [
    bigquery.SchemaField("date_month", "DATE", mode="REQUIRED"),
    bigquery.SchemaField("report_created_ts", "TIMESTAMP", mode="REQUIRED"),
    bigquery.SchemaField("report_data", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("report_analysis", "STRING", mode="REQUIRED"),
  ]

  # Create the table with the new schema
  table = bigquery.Table(table_ref, schema=schema)
  table = client.create_table(table, exists_ok=True)

  rows_to_insert = [
    (date_month, datetime.now(), report_data, report_analysis)
  ]

  errors = client.insert_rows(table, rows_to_insert)
  if errors == []:
    print(f"Report for {date_month} successfully written to BigQuery")
  else:
    print(f"Errors occurred while inserting rows: {errors}")


# Main execution flow
def main():
  # Initialize clients
  bigquery_client, llm = initialize_clients()

  # Fetch data
  data, account_group_data, transactions_data = get_pl_data(bigquery_client)

  # Process data
  report_data, second_last_month, third_last_month, unique_months, current_year, previous_year = process_data(data, account_group_data, transactions_data)

  # Prepare account group details
  current_month_groups = {category: get_account_group_details(account_group_data, transactions_data, category, second_last_month)
              for category in ['REVENUE', 'DIRECTCOSTS', 'EXPENSE']}
  account_group_details = prepare_account_group_details(current_month_groups)

  # Analyze cost changes
  cost_change_analysis = analyze_cost_changes(transactions_data, 'Direct Costs', second_last_month)

  # Generate financial data string
  financial_data = generate_financial_data_string(report_data, second_last_month, third_last_month, current_year, previous_year)

  # Generate analysis prompt
  analysis_prompt = generate_analysis_prompt(second_last_month.strftime('%B %Y'), financial_data, account_group_details, cost_change_analysis)

  # Generate analysis using LLM
  try:
    report_analysis_html = llm(analysis_prompt.format(
      month=second_last_month.strftime('%B %Y'),
      financial_data=financial_data,
      account_group_details=account_group_details,
      cost_change_analysis=cost_change_analysis
    ))
  except Exception as e:
    report_analysis_html = f"Error generating analysis: {html.escape(str(e))}"

  # Generate report HTML
  report_data_html = generate_report_html(report_data, second_last_month, third_last_month, unique_months, current_year, previous_year)
  report_data_html = re.sub(r'\*\*(.+?)\*\*', r'<b>\1</b>', report_data_html)

  # Generate full HTML output for display
  full_html_output = generate_full_html_output(
    report_analysis_html.replace("**", ""),
    report_data_html.replace("**", "")
  )

  # Display the full report
  display(HTML(full_html_output))

  # Write report to BigQuery
  write_report_to_bigquery(bigquery_client, second_last_month.date(), report_data_html.replace("**", ""), report_analysis_html.replace("**", ""))

if __name__ == "__main__":
  main()


<ipython-input-44-0db75cc68701>:582: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  growth_rates = grouped.pct_change(axis=1).mean(axis=1)


Category,Aug 2024,% Change MoM,Jul 2024,Jun 2024,YTD 2024,% Change YTD,YTD 2023
Turnover,"£11,419.79",246.95%,"£3,291.45","£3,679.17","£21,890.42",525.44%,"£3,500.00"
Direct Costs,"£-1,250.00",nan%,£nan,£nan,"£-1,250.00",nan%,£nan
Direct Costs as % of Sales,-10.95%,,nan%,nan%,-5.71%,,nan%
Gross Profit,"£10,169.79",208.98%,"£3,291.45","£3,679.17","£20,640.42",489.73%,"£3,500.00"
Gross Profit as % of Sales,89.05%,,100.00%,100.00%,94.29%,,100.00%
Overheads,"£-8,541.52",-58.70%,"£-5,382.09","£-1,282.60","£-15,682.50",-158.96%,"£-6,056.00"
Overheads as % of Sales,-74.80%,,-163.52%,-34.86%,-71.64%,,-173.03%
Net Profit,"£1,628.27",177.88%,"£-2,090.64","£2,396.57","£4,957.92",293.97%,"£-2,556.00"
Net Profit as % of Sales,14.26%,,-63.52%,65.14%,22.65%,,-73.03%
Taxation,£407.07,177.88%,£-522.66,£599.14,"£1,239.48",293.97%,£-639.00


Report for 2024-08-01 successfully written to BigQuery
